In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.nuscenes import RadarPointCloud
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix

import operator
import time

import cv2
import numpy as np
import open3d as o3d
from mayavi import mlab
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

from pyquaternion import Quaternion

from transformers import AutoImageProcessor, DetrForObjectDetection
import torch
from PIL import Image,ImageDraw
import requests


In [ ]:
nusc = NuScenes(version='v1.0-mini', 
                dataroot='/home/zdhjs-05/myGitHubCode/v1.0-mini', verbose=True)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")


In [ ]:
device = torch.device("cuda")
model.to(device)
model.device

In [ ]:
scene = nusc.scene[0] 

# 获取该场景的first token
cur_sample_info = nusc.get('sample', scene['first_sample_token'])
sensor_data_token = nusc.get('sample_data', cur_sample_info['data']['CAM_FRONT'])
img_path_ = nusc.get_sample_data_path(cur_sample_info['data']['CAM_FRONT'])
image = Image.open(img_path_)
inputs = image_processor(images=image, return_tensors="pt")
inputs.to(device)
outputs = model(**inputs)

target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.4, target_sizes=target_sizes)[0]

draw = ImageDraw.Draw(image)
img = cv2.imread(img_path_)


for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(box)
    draw.rectangle(box, outline="#FF0000", width=2)
    
    img = cv2.rectangle(img, (int(box[0]), int(box[1])),
                             (int(box[2]),int(box[3])), (255, 0,0))
cv2.imshow('xxx', img)
cv2.waitKey(1000)
cv2.destroyAllWindows()
# image.show()